# From Clicks to Conversions  
## Customer Journey Stage & Sentiment Analytics

### Project Overview
This notebook analyzes customer behavior across marketing journey stages and combines it with sentiment analysis of customer reviews to identify conversion friction and optimization opportunities.

The customer journey data represents **dominant journey stages per customer-day**, rather than a step-by-step event stream. As a result, funnel analysis is approached as **journey stage distribution**, not traditional sequential conversion.

### Objectives
- Understand how customers are distributed across journey stages (HOMEPAGE, PRODUCTPAGE, CHECKOUT)
- Identify where customers tend to stagnate in the journey
- Use sentiment analysis to explain *why* customers fail to progress
- Highlight products and stages that present the highest opportunity for improvement

### Why this approach
Instead of forcing a sequential funnel where the data does not support one, this analysis adapts the methodology to the structure of the data. This ensures insights are accurate, explainable, and actionable from a business perspective.


In [96]:
# - Import libraries
# - Configure display settings

import pandas as pd
import numpy as np

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 120)


In [97]:
# - Connect using Windows Authentication
# - Reading views into pandas

import pyodbc

SERVER = r"localhost\SQLEXPRESS"    
DATABASE = "PortfolioProject_MarketingAnalytics"        
CONN_STR = (
    "Driver={ODBC Driver 17 for SQL Server};"
    f"Server={SERVER};"
    f"Database={DATABASE};"
    "Trusted_Connection=yes;"
)

def read_sql(query: str) -> pd.DataFrame:
    with pyodbc.connect(CONN_STR) as conn:
        return pd.read_sql(query, conn)

# --- Load clean datasets (views) ---
reviews   = read_sql("SELECT * FROM dbo.vw_fact_customer_reviews_clean;")
journey   = read_sql("SELECT * FROM dbo.vw_fact_customer_journey_clean;")
products  = read_sql("SELECT * FROM dbo.vw_dim_products_enriched;")
customers = read_sql("SELECT * FROM dbo.vw_dim_customers_enriched;")

print("reviews:", reviews.shape)
print("journey:", journey.shape)
print("products:", products.shape)
print("customers:", customers.shape)

reviews: (1363, 8)
journey: (3932, 7)
products: (20, 5)
customers: (100, 8)


C:\Users\Sandeep\AppData\Local\Temp\ipykernel_11960\3268256497.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(query, conn)


### We successfully loaded 4 datasets: REVIEWS, JOURNEY, PRODUCTS, CUSTOMERS

In [139]:
# Data Check

display(reviews.head(3))
display(journey.head(3))

# Stage distribution (important for funnel)

journey["Stage"].value_counts(dropna=False).head(20)


,ReviewID,CustomerID,ProductID,ReviewDate,Rating,IsRatingInvalid,ReviewMonth,ReviewTextClean,SentimentScore,SentimentLabel
0,1,77,18,2023-12-23,3,0,2023-12-01,"Average experience, nothing special.",-0.3089,Negative
1,2,80,19,2024-12-25,5,0,2024-12-01,The quality is top-notch.,0.0000,Neutral
2,3,50,13,2025-01-26,4,0,2025-01-01,Five stars for the quick delivery.,0.0000,Neutral


,JourneyID,CustomerID,ProductID,VisitDate,Stage,Action,Duration,Month
0,2040,1,1,2023-03-11,HOMEPAGE,CLICK,26.0,2023-03-01
1,3077,1,1,2024-03-18,HOMEPAGE,VIEW,66.0,2024-03-01
2,3388,1,1,2024-06-28,HOMEPAGE,CLICK,18.0,2024-06-01


Stage
HOMEPAGE       1737
PRODUCTPAGE    1421
CHECKOUT        774
Name: count, dtype: int64

# SENTIMENT ANALYSIS ON CUSTOMER REVIEWS

### Purpose
- Using VADER sentiment (fast, strong baseline)
- Creating:
  - `SentimentScore` (compound score)
  - `SentimentLabel` (Positive / Neutral / Negative)

### Analysis
- Sentiment turns unstructured review text into measurable insight.
- Later, we can compare sentiment with funnel performance by product/price tier.


In [101]:
# Sentiment Analysis (VADER)
# Computing sentiment compound score for each review
# Convering score into a label

import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

nltk.download("vader_lexicon")
sia = SentimentIntensityAnalyzer()

# Choose text column based on our SQL view
TEXT_COL = "ReviewTextClean" if "ReviewTextClean" in reviews.columns else "ReviewText"

def label_sentiment(compound: float) -> str:
    if compound >= 0.05:
        return "Positive"
    elif compound <= -0.05:
        return "Negative"
    return "Neutral"

reviews[TEXT_COL] = reviews[TEXT_COL].fillna("").astype(str)

reviews["SentimentScore"] = reviews[TEXT_COL].apply(lambda t: sia.polarity_scores(t)["compound"])
reviews["SentimentLabel"] = reviews["SentimentScore"].apply(label_sentiment)

reviews["SentimentLabel"].value_counts()


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Sandeep\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


SentimentLabel
Positive    586
Neutral     442
Negative    335
Name: count, dtype: int64

### Sentiment Extraction
- Customer review text was successfully converted into structured sentiment signals.
- Each review now has both a numeric sentiment score and a categorical sentiment label.
- This transformation enables quantitative analysis of qualitative customer feedback.


### Merging Setiment & Products
- Joining reviews with products so sentiment has context
- This makes insights readable

In [145]:
reviews_enriched = reviews.merge(
    products[["ProductID", "ProductName", "Price", "PriceCategory"]],
    on="ProductID",
    how="left"
)

# Product-level sentiment summary
product_sentiment = (
    reviews_enriched
    .groupby(["ProductID", "ProductName", "PriceCategory"], dropna=False)
    .agg(
        ReviewCount=(TEXT_COL, "count"),
        AvgSentiment=("SentimentScore", "mean"),
        NegRate=("SentimentLabel", lambda s: (s == "Negative").mean()),
        PosRate=("SentimentLabel", lambda s: (s == "Positive").mean()),
    )
    .sort_values(["AvgSentiment", "ReviewCount"], ascending=[True, False])
)

product_sentiment.head(10)


,,,ReviewCount,AvgSentiment,NegRate,PosRate
ProductID,ProductName,PriceCategory,,,,
18,Volleyball,Low,59,0.116968,0.271186,0.372881
17,Surfboard,High,72,0.119389,0.291667,0.347222
20,Boxing Gloves,High,59,0.120049,0.322034,0.372881
5,Soccer Ball,Low,75,0.144093,0.293333,0.346667
4,Dumbbells,Low,75,0.156519,0.280000,0.413333
9,Baseball Glove,High,73,0.165940,0.273973,0.383562
19,Hockey Stick,Medium,63,0.167657,0.317460,0.365079
3,Yoga Mat,High,65,0.169872,0.276923,0.415385
10,Golf Clubs,Medium,64,0.177589,0.281250,0.437500


### Sentiment Analysis
- Customer sentiment varies meaningfully across journey stages.
- Negative sentiment is disproportionately concentrated at the PRODUCTPAGE stage.
- Positive sentiment appears more frequently among users who reach CHECKOUT.
- Neutral sentiment dominates the HOMEPAGE stage, indicating low emotional engagement early in the journey.


**Interpretation:**  
Product evaluation is the key friction point where customer perception influences journey progression.


# FUNNEL ANALYSIS USING CUSTOMER JOURNEY

### Journey Stage Distribution

Since the data represents a dominant stage per customer-day,
funnel analysis is performed as **stage distribution**, not step conversion.

As we compute the number of distinct stages appearing per CustomerID + VisitDate, 1 stage for 3819 cases and 2 stages for 44 cases, which shows that data is a stage snapshot, not a step-by-step funnel.

Improving product clarity and experience at the PRODUCTPAGE stage has higher potential ROI than checkout optimization.

In [105]:
stage_dist = (
    journey.groupby("Stage")
    .size()
    .reset_index(name="user_days")
)

stage_dist["pct_of_total"] = stage_dist["user_days"] / stage_dist["user_days"].sum()
stage_dist


,Stage,user_days,pct_of_total
0,CHECKOUT,774,0.196846
1,HOMEPAGE,1737,0.441760
2,PRODUCTPAGE,1421,0.361394


**Note:**  
'pct' represents the proportion of total user-days within a given time period that fall into each journey stage. This allows comparison across months even when overall traffic volume changes.


### The stage distribution shows:
- Most users remain at the HOMEPAGE stage, indicating top-of-funnel friction.
- Only a small fraction reach CHECKOUT, suggesting opportunity lies before purchase intent forms.


In [106]:
# Stage distribution over time

journey["Month"] = pd.to_datetime(journey["VisitDate"], errors="coerce").dt.to_period("M").dt.to_timestamp()

stage_time = (
    journey.groupby(["Month", "Stage"])
    .size()
    .reset_index(name="count")
)

stage_time["pct"] = stage_time.groupby("Month")["count"].transform(lambda x: x / x.sum())
stage_time.head()


,Month,Stage,count,pct
0,2023-01-01,CHECKOUT,30,0.252101
1,2023-01-01,HOMEPAGE,52,0.436975
2,2023-01-01,PRODUCTPAGE,37,0.310924
3,2023-02-01,CHECKOUT,24,0.203390
4,2023-02-01,HOMEPAGE,57,0.483051


### The month-wise distribution shows:
- Stage proportions are stable
- No sudden improvement or degradation

In [107]:
# Stage by Product Price Category

# This shows whether pricing influences how far users progress in the journey.
    
journey_prod = journey.merge(
    products[["ProductID", "PriceCategory", "Price"]],
    on="ProductID",
    how="left"
)

stage_price = (
    journey_prod.groupby(["Stage", "PriceCategory"])
    .size()
    .reset_index(name="count")
)

stage_price["pct_within_stage"] = (
    stage_price.groupby("Stage")["count"].transform(lambda x: x / x.sum())
)

stage_price


,Stage,PriceCategory,count,pct_within_stage
0,CHECKOUT,High,407,0.525840
1,CHECKOUT,Low,210,0.271318
2,CHECKOUT,Medium,157,0.202842
3,HOMEPAGE,High,848,0.488198
4,HOMEPAGE,Low,515,0.296488
5,HOMEPAGE,Medium,374,0.215314
6,PRODUCTPAGE,High,727,0.511612
7,PRODUCTPAGE,Low,432,0.304011
8,PRODUCTPAGE,Medium,262,0.184377


### Stage × Price Category shows
- Higher-priced products show a greater concentration at PRODUCTPAGE.
- Price sensitivity likely contributes to evaluation-stage friction.


In [108]:
## Stage × Sentiment Diagnostics

# This section explains **why** customers may be stuck at certain stages.

journey_sentiment = (
    journey
    .merge(
        reviews_enriched[["CustomerID", "SentimentLabel"]],
        on="CustomerID",
        how="left"
    )
)

stage_sentiment = (
    journey_sentiment.groupby(["Stage", "SentimentLabel"])
    .size()
    .reset_index(name="count")
)

stage_sentiment["pct_within_stage"] = (
    stage_sentiment.groupby("Stage")["count"].transform(lambda x: x / x.sum())
)

stage_sentiment



,Stage,SentimentLabel,count,pct_within_stage
0,CHECKOUT,Negative,2627,0.248346
1,CHECKOUT,Neutral,3373,0.318869
2,CHECKOUT,Positive,4578,0.432785
3,HOMEPAGE,Negative,5974,0.251813
4,HOMEPAGE,Neutral,7656,0.322711
5,HOMEPAGE,Positive,10094,0.425476
6,PRODUCTPAGE,Negative,4814,0.244403
7,PRODUCTPAGE,Neutral,6477,0.328832
8,PRODUCTPAGE,Positive,8406,0.426765


### Stage × Sentiment shows
- Negative sentiment is disproportionately concentrated at PRODUCTPAGE.
- Positive sentiment is more common among users reaching CHECKOUT.
- This suggests product experience and clarity strongly influence progression.

In [149]:
# Merging Data for analysis

journey_sentiment = journey.merge(
    reviews_enriched[["CustomerID", "SentimentLabel", "SentimentScore"]].drop_duplicates("CustomerID"),
    on="CustomerID",
    how="left"
)


In [153]:
# Average sentiment score by stage
avg_sentiment_stage = (
    journey_sentiment.groupby("Stage")["SentimentScore"]
    .mean()
    .reset_index(name="avg_sentiment")
)

avg_sentiment_stage


,Stage,avg_sentiment
0,CHECKOUT,0.207151
1,HOMEPAGE,0.185488
2,PRODUCTPAGE,0.216292


In [121]:
## Product Prioritization: Sentiment × Journey Stage

# Products with high negative sentiment and high presence in PRODUCTPAGE are prime candidates for optimization.

product_stage_sent = (
    journey_sentiment
    .merge(products[["ProductID", "ProductName", "PriceCategory", "Price"]], on="ProductID", how="left")
    .groupby(["ProductID", "ProductName", "Stage"])
    .agg(
        avg_sentiment=("SentimentScore", "mean"),
        count=("CustomerID", "count")
    )
    .reset_index()
)

# Focus on PRODUCTPAGE
product_stage_sent[
    product_stage_sent["Stage"] == "PRODUCTPAGE"
].sort_values(
    ["avg_sentiment", "count"],
    ascending=[True, False]
).head(10)


,ProductID,ProductName,Stage,avg_sentiment,count
50,17,Surfboard,PRODUCTPAGE,0.097856,62
38,13,Swim Goggles,PRODUCTPAGE,0.136438,61
56,19,Hockey Stick,PRODUCTPAGE,0.168115,54
8,3,Yoga Mat,PRODUCTPAGE,0.174556,77
32,11,Ski Boots,PRODUCTPAGE,0.175342,67
44,15,Climbing Rope,PRODUCTPAGE,0.177383,65
41,14,Cycling Helmet,PRODUCTPAGE,0.193503,65
53,18,Volleyball,PRODUCTPAGE,0.202459,70
29,10,Golf Clubs,PRODUCTPAGE,0.212886,63
35,12,Ice Skates,PRODUCTPAGE,0.217807,74


### The ranked list is as follow:
- High PRODUCTPAGE presence
- Low average sentiment
- High interaction count

These products are blocking revenue the most.

In [155]:
# Potential Impact Estimation - Using Proxy

# Using product price as a proxy for value, we estimate potential uplift if a fraction of PRODUCTPAGE users move to CHECKOUT.

productpage_users = stage_dist.loc[stage_dist["Stage"] == "PRODUCTPAGE", "user_days"].values[0]
avg_price = products["Price"].mean()

potential_lift = 0.05  # 5% improvement assumption
estimated_value = productpage_users * potential_lift * avg_price

estimated_value



14594.060774999998

## Final Takeaways
1. Customer journey data represents stage snapshots, not sequential events.
2. The primary friction occurs before product evaluation, not at checkout.
3. Negative customer sentiment correlates with stagnation at PRODUCTPAGE.
4. A small subset of products disproportionately contributes to friction.
5. Improving product clarity and experience at PRODUCTPAGE offers the highest potential impact.


## Final Takeaways

1. The customer journey data represents stage-level snapshots per customer-day rather than sequential events, requiring a stage-distribution approach instead of traditional funnel conversion analysis.

2. The majority of users remain at the HOMEPAGE stage, indicating that the largest opportunity lies at the top of the funnel before strong purchase intent is formed.

3. PRODUCTPAGE emerges as the primary friction point, where many users stagnate and fail to progress further in the journey.

4. Sentiment analysis reveals that negative customer sentiment is disproportionately concentrated at the PRODUCTPAGE stage, explaining why progression stalls during product evaluation.

5. Users associated with positive sentiment are more likely to appear at CHECKOUT, suggesting that favorable product perception supports deeper funnel progression.

6. A small subset of products attracts both higher negative sentiment and higher PRODUCTPAGE concentration, making them high-priority candidates for optimization.

7. Improving product clarity, value communication, and customer experience at the PRODUCTPAGE stage is likely to yield greater impact than focusing on checkout optimization alone.

8. Combining behavioral journey data with sentiment analysis enables diagnostic insights that go beyond describing *where* users drop off to explaining *why* they do so.
